In [ ]:
import kagglehub
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import pandas as pd
from lightgbm import LGBMClassifier

path = kagglehub.dataset_download("chethuhn/network-intrusion-dataset")

print("Path to dataset files:", path)
path = str(path)

print(os.listdir(path))

c:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\abdul\.cache\kagglehub\datasets\chethuhn\network-intrusion-dataset\versions\1
['Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', 'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv', 'Friday-WorkingHours-Morning.pcap_ISCX.csv', 'Monday-WorkingHours.pcap_ISCX.csv', 'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv', 'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', 'Tuesday-WorkingHours.pcap_ISCX.csv', 'Wednesday-workingHours.pcap_ISCX.csv']


In [2]:
ddos_data = pd.read_csv(path + "/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
normal_data = pd.read_csv(path + "/Monday-WorkingHours.pcap_ISCX.csv")

df = pd.concat([ddos_data, normal_data], ignore_index=True)
df.columns = df.columns.str.strip()

df.replace([np.inf, -np.inf], np.nan, inplace=True)


df.dropna(inplace=True)


In [3]:
selected_features = [
    "Total Fwd Packets", "Total Backward Packets", "Total Length of Fwd Packets",
    "Total Length of Bwd Packets", "Fwd Packet Length Mean", "Bwd Packet Length Mean",
    "Packet Length Mean", "Flow Bytes/s", "Flow Packets/s", "Fwd Packets/s", "Bwd Packets/s",
    "Subflow Fwd Packets", "Subflow Bwd Packets", "Flow Duration", "Flow IAT Mean",
    "Fwd IAT Mean", "Bwd IAT Mean", "Active Mean", "Idle Mean", "SYN Flag Count",
    "ACK Flag Count", "FIN Flag Count", "RST Flag Count", "PSH Flag Count",
    "Init_Win_bytes_forward", "Init_Win_bytes_backward", "min_seg_size_forward"
]

X = df[selected_features]
y = df["Label"].apply(lambda x: 1 if "DDoS" in str(x) else 0)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
lgbm_model = LGBMClassifier(n_estimators=200, boosting_type='gbdt', random_state=42)

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
model = LogisticRegression()

In [6]:
model.fit(X_train, y_train)

LogisticRegression()

In [7]:
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [8]:
xgb_model.fit(X_train, y_train)

c:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [23:59:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [9]:
lgbm_model.fit(X_train, y_train)

c:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


[LightGBM] [Info] Number of positive: 102453, number of negative: 501700
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025813 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5383
[LightGBM] [Info] Number of data points in the train set: 604153, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.169581 -> initscore=-1.588598
[LightGBM] [Info] Start training from score -1.588598
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMClassifier(n_estimators=200, random_state=42)

In [10]:
y_pred = model.predict(X_test)
y_pred_rf = rf_model.predict(X_test)
y_pred_xgb = xgb_model.predict(X_test)
y_pred_lgbm = lgbm_model.predict(X_test)

c:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [11]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.9843

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    125467
           1       0.98      0.93      0.95     25572

    accuracy                           0.98    151039
   macro avg       0.98      0.96      0.97    151039
weighted avg       0.98      0.98      0.98    151039


Confusion Matrix:
[[125008    459]
 [  1907  23665]]


In [12]:
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.9998940670952535
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    125467
           1       1.00      1.00      1.00     25572

    accuracy                           1.00    151039
   macro avg       1.00      1.00      1.00    151039
weighted avg       1.00      1.00      1.00    151039



In [13]:
print("LightGBM Accuracy:", accuracy_score(y_test, y_pred_lgbm))
print(classification_report(y_test, y_pred_lgbm))

LightGBM Accuracy: 0.9999404127410801
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    125467
           1       1.00      1.00      1.00     25572

    accuracy                           1.00    151039
   macro avg       1.00      1.00      1.00    151039
weighted avg       1.00      1.00      1.00    151039



In [14]:
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))


XGBoost Accuracy: 0.9999536543541734
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    125467
           1       1.00      1.00      1.00     25572

    accuracy                           1.00    151039
   macro avg       1.00      1.00      1.00    151039
weighted avg       1.00      1.00      1.00    151039

